## Edge Coloring Assessment

In [1]:
import glob
import json
import random
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.filters import sobel_h, sobel_v, gaussian

Functions to detect and color edges.

In [2]:
def edge_detection(img):
    """Returns the magnitude and direction of the edges in img"""
    
    img_gradient_x = sobel_v(img)
    img_gradient_y = sobel_h(img)
    img_gradient_magnitude = ((img_gradient_x ** 2) + (img_gradient_y ** 2)) ** (1/2)
    img_gradient_direction = np.arctan2(img_gradient_x, img_gradient_y)
    return img_gradient_x, img_gradient_y, img_gradient_magnitude, img_gradient_direction

def edge_color(img_edge_magnitude, img_edge_direction, cmap):
    """Colors the edges of the image using their direction and a colormap."""
    
    img_shape = img_edge_magnitude.shape
    img_edge_direction = img_edge_direction + abs(img_edge_direction.min())
    img_edge_direction_max = img_edge_direction.max()
    color_range = plt.cm.get_cmap(cmap)
    black_pixel = np.array([0, 0, 0])
    result = np.zeros((img_shape[0], img_shape[1], 3))
    for i in range(img_shape[0]):
        for j in range(img_shape[1]):
            if img_edge_magnitude[i, j] == 1:
                # if edge is present, select a color based on the edge direction
                color_index = img_edge_direction[i, j] / img_edge_direction_max
                result[i, j] = color_range(color_index)[:3]
            else:   # if edge is not present, just add a black pixel
                result[i, j] = black_pixel
    return result

### Edge Detection

In [3]:
sigma = 1
threshold = 0.1

filenames = glob.glob('experiments/original/*')
for filename in filenames:
    img = Image.open(filename)
    img = img.convert("L")   # convert to grayscale
    img = np.array(img)
    img = img/255
    img_blurred = gaussian(img, sigma)
    img_gradient_x, img_gradient_y, img_edge_magnitude, img_edge_direction = edge_detection(img_blurred)
    img_edge_magnitude_thresholded = (img_edge_magnitude > threshold)
    img_edge_magnitude = Image.fromarray((img_edge_magnitude_thresholded * 255).astype('uint8'))
    img_edge_magnitude.save('experiments/edges_magnitude/{}.jpg'.format(filename[21:-4]))

### Edge Coloring

In [5]:
cmaps = {}
cmaps['sequential'] = ['viridis', 'plasma', 'inferno', 'magma', 'cividis',
                       'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                       'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                       'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
cmaps['diverging'] = ['PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
                      'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']
cmaps['cyclic'] = ['twilight', 'twilight_shifted', 'hsv']
cmaps['qualitative'] = ['Pastel1', 'Pastel2', 'Paired', 'Accent',
                        'Dark2', 'Set1', 'Set2', 'Set3',
                        'tab10', 'tab20', 'tab20b', 'tab20c']

sigma = 1
threshold = 0.1
random.seed(0)
filenames = glob.glob('experiments/original/*')
for i, filename in enumerate(filenames):
    print("Edge coloring {} out of {} files".format(i+1, len(filenames)))
    img = Image.open(filename)
    img = img.convert("L")   # convert to grayscale
    img = np.array(img)
    img = img/255
    img_blurred = gaussian(img, sigma)
    img_gradient_x, img_gradient_y, img_edge_magnitude, img_edge_direction = edge_detection(img_blurred)
    img_edge_magnitude_thresholded = (img_edge_magnitude > threshold)
    img_edge_magnitude = Image.fromarray((img_edge_magnitude_thresholded * 255).astype('uint8'))
    for cmap_type, cmap_names in cmaps.items():
        cmap = random.choice(cmap_names)
        img_edge_colored = edge_color(img_edge_magnitude_thresholded, img_edge_direction, cmap)
        img_edge_colored = Image.fromarray((img_edge_colored * 255).astype('uint8'))
        img_edge_colored.save('experiments/edges_colored/{}/{}.jpg'.format(cmap_type, filename[21:-4]))

Edge coloring 1 out of 100 files
Edge coloring 2 out of 100 files
Edge coloring 3 out of 100 files
Edge coloring 4 out of 100 files
Edge coloring 5 out of 100 files
Edge coloring 6 out of 100 files
Edge coloring 7 out of 100 files
Edge coloring 8 out of 100 files
Edge coloring 9 out of 100 files
Edge coloring 10 out of 100 files
Edge coloring 11 out of 100 files
Edge coloring 12 out of 100 files
Edge coloring 13 out of 100 files
Edge coloring 14 out of 100 files
Edge coloring 15 out of 100 files
Edge coloring 16 out of 100 files
Edge coloring 17 out of 100 files
Edge coloring 18 out of 100 files
Edge coloring 19 out of 100 files
Edge coloring 20 out of 100 files
Edge coloring 21 out of 100 files
Edge coloring 22 out of 100 files
Edge coloring 23 out of 100 files
Edge coloring 24 out of 100 files
Edge coloring 25 out of 100 files
Edge coloring 26 out of 100 files
Edge coloring 27 out of 100 files
Edge coloring 28 out of 100 files
Edge coloring 29 out of 100 files
Edge coloring 30 out of

### Neural Image Assessment (NIMA) Evaluation

In [6]:
%cd image-quality-assessment
!ls

/Users/shouvikmani/Documents/edge-colorizer/image-quality-assessment
CONTRIBUTING.md README.md       entrypoints     predict
Dockerfile.cpu  _readme         experiments     src
Dockerfile.gpu  contrib         mkdocs          train-ec2
LICENSE         data            models          train-local


In [7]:
nima_aesthetic_scores = !./predict  --docker-image nima-cpu --base-model-name MobileNet --weights-file $(pwd)/models/MobileNet/weights_mobilenet_aesthetic_0.07.hdf5 --image-source $(pwd)/../experiments/edges_magnitude
nima_aesthetic_scores = ''.join(nima_aesthetic_scores[11:])
nima_aesthetic_scores = json.loads(nima_aesthetic_scores)
nima_aesthetic_scores_df = pd.DataFrame(nima_aesthetic_scores)
mean_nima_aesthetic_score = nima_aesthetic_scores_df['mean_score_prediction'].mean()
print("Mean NIMA aesthetic score, black & white edges:", mean_nima_aesthetic_score)

Mean NIMA aesthetic score, black & white edges: 5.2768248715625745


In [50]:
for cmap_type in cmaps:
    nima_aesthetic_scores = !./predict  --docker-image nima-cpu --base-model-name MobileNet --weights-file $(pwd)/models/MobileNet/weights_mobilenet_aesthetic_0.07.hdf5 --image-source $(pwd)/../experiments/edges_colored/{cmap_type}
    nima_aesthetic_scores = ''.join(nima_aesthetic_scores[11:])
    nima_aesthetic_scores = json.loads(nima_aesthetic_scores)
    nima_aesthetic_scores_df = pd.DataFrame(nima_aesthetic_scores)
    mean_nima_aesthetic_score = nima_aesthetic_scores_df['mean_score_prediction'].mean()
    print("Mean NIMA aesthetic score, colored edges ({} color map):".format(cmap_type), mean_nima_aesthetic_score)

Mean NIMA aesthetic score, colored edges (sequential color map): 5.2434288593710106
Mean NIMA aesthetic score, colored edges (diverging color map): 5.237948914232568
Mean NIMA aesthetic score, colored edges (cyclic color map): 5.266123633883767
Mean NIMA aesthetic score, colored edges (qualitative color map): 5.242835082153793


In [8]:
nima_technical_scores = !./predict  --docker-image nima-cpu --base-model-name MobileNet --weights-file $(pwd)/models/MobileNet/weights_mobilenet_technical_0.11.hdf5 --image-source $(pwd)/../experiments/edges_magnitude
nima_technical_scores = ''.join(nima_technical_scores[11:])
nima_technical_scores = json.loads(nima_technical_scores)
nima_technical_scores_df = pd.DataFrame(nima_technical_scores)
mean_nima_technical_score = nima_technical_scores_df['mean_score_prediction'].mean()
print("Mean NIMA technical score, black & white edges:", mean_nima_technical_score)

Mean NIMA technical score, black & white edges: 4.49039006665349


In [59]:
for cmap_type in cmaps:
    nima_technical_scores = !./predict  --docker-image nima-cpu --base-model-name MobileNet --weights-file $(pwd)/models/MobileNet/weights_mobilenet_technical_0.11.hdf5 --image-source $(pwd)/../experiments/edges_colored/{cmap_type}
    nima_technical_scores = ''.join(nima_technical_scores[11:])
    nima_technical_scores = json.loads(nima_technical_scores)
    nima_technical_scores_df = pd.DataFrame(nima_technical_scores)
    mean_nima_technical_score = nima_technical_scores_df['mean_score_prediction'].mean()
    print("Mean NIMA technical score, colored edges ({} color map):".format(cmap_type), mean_nima_technical_score)

Mean NIMA technical score, colored edges (sequential color map): 4.614252245128155
Mean NIMA technical score, colored edges (diverging color map): 4.665977361239493
Mean NIMA technical score, colored edges (cyclic color map): 4.655434283129871
Mean NIMA technical score, colored edges (qualitative color map): 4.660255505368114
